### 데이터 불러오기

In [6]:
import duckdb

# DuckDB에 연결
# 절대 경로 지정 필요 (이유 불문)
conn = duckdb.connect(database='/home/hooniegit/git/personal/duck_and_boxoffice/database/memory', read_only=False)  
cursor = conn.cursor()

In [12]:
# 쿼리 실행
# 컬럼 확인하기
cursor.execute("""
               SELECT column_name
               FROM information_schema.columns
               WHERE table_name = 'box_seoul';
               """)

# 결과 가져오기
results = cursor.fetchall()
columns = [result[0] for result in results]

In [7]:
# 쿼리 실행
# 서울 2018~2023 테이블
# 주말 데이터 가져오기
cursor.execute("""
               SELECT *
               FROM box_seoul
               WHERE EXTRACT(DOW FROM CAST(SUBSTRING(date FROM 1 FOR 4) || '-' || SUBSTRING(date FROM 6 FOR 2) || '-' || SUBSTRING(date FROM 9 FOR 2) AS DATE)) IN (6, 0);
               """)

# 결과 가져오기
result_weekend = cursor.fetchall()

In [8]:
# 쿼리 실행
# 서울 2018~2023 테이블
# 주중 데이터 가져오기
cursor.execute("""
               SELECT *
               FROM box_seoul
               WHERE EXTRACT(DOW FROM CAST(SUBSTRING(date FROM 1 FOR 4) || '-' || SUBSTRING(date FROM 6 FOR 2) || '-' || SUBSTRING(date FROM 9 FOR 2) AS DATE)) NOT IN (6, 0);
               """)

# 결과 가져오기
result_weekday = cursor.fetchall()

In [24]:
# 커넥터 종료
conn.close()

### 데이터 분석하기

In [16]:
import pandas as pd

# 데이터프레임 생성
df_weekday = pd.DataFrame(columns=columns, data=result_weekday)
df_weekend = pd.DataFrame(columns=columns, data=result_weekend)

In [18]:
df_weekday.head()

,date,rank,movie_nm,movie_open,sales_amount,sales_share,sales_inten,sales_change,sales_acc,audi_cnt,audi_inten,audi_change,audi_acc,scrn_cnt,show_cnt,loc_code
0,2018-02-15,1,블랙 팬서,2018-02-14,1451937500,58.1,14033300,1,2904675700,154900,-17481.0,-10.1,328792,314,1636,0105001
1,2018-02-15,2,골든슬럼버,2018-02-14,360629000,14.4,49190900,16,727944300,41625,1774.0,4.5,87057,162,704,0105001
2,2018-02-15,3,조선명탐정: 흡혈괴마의 비밀,2018-02-08,251797300,10.1,137464700,120,2456907445,29266,14485.0,98,297182,137,454,0105001
3,2018-02-15,4,흥부: 글로 세상을 바꾼 자,2018-02-14,146467800,5.9,51045700,54,273207900,16693,4456.0,36.4,32780,114,403,0105001
4,2018-02-15,5,코코,2018-01-11,61988400,2.5,32011200,107,8204294016,7207,3351.0,86.9,981295,83,127,0105001


In [32]:
# 서울 5년 통산 평일 / 주말 영화 관람객수
audi_weekday_total = (df_weekday['audi_cnt'].sum())
audi_weekend_total = (df_weekend['audi_cnt'].sum())

# 서울 5년 통산 평일 / 주말 영화 관람객수 - 요일 평균
weekday_cnt = df_weekday['date'].count()
weekend_cnt = df_weekend['date'].count()

audi_weekday_avg = f"{(df_weekday['audi_cnt'].sum()) / weekday_cnt:.1f}"
audi_weekend_avg = f"{(df_weekend['audi_cnt'].sum()) / weekend_cnt:.1f}"

In [33]:
print(audi_weekday_total, audi_weekend_total)
print(audi_weekday_avg, audi_weekend_avg)

102012486 81892412
7089.1 14291.9
